In [15]:
import pandas as pd
import numpy as np
import camelot
import tempfile
import fitz
import re

In [16]:
SECTION_RANGES = {
    'PRE-PRODUCTION | WRAP LABOR': range(0, 51),
    'SHOOTING LABOR': range(51, 101),
    'PRE-PRODUCTION | WRAP EXPENSES': range(101, 114),
    'LOCATION AND TRAVEL': range(114, 139),
    'MAKEUP, WARDROBE, AND ANIMALS': range(140, 151),
    'STUDIO | STAGE RENTAL / EXPENSES': range(151, 167),
    'ART DEPARTMENT LABOR': range(168, 181),
    'ART DEPARTMENT EXPENSES': range(181, 193),
    'EQUIPMENT COSTS': range(193, 210),
    'FILMSTOCK, DEVELOP AND PRINT': range(211, 217),
    'MISCELLANEOUS': range(217, 227),
    'DIRECTOR | CREATIVE FEES': range(227, 234),
    'TALENT LABOR': range(234, 271),
    'TALENT EXPENSES': range(271, 277),
    'POST PRODUCTION LABOR': range(277, 282),
    'EDITORIAL | FINISHING | POST PRODUCTION': range(282, 329)
}


def get_section_from_line(ln:int) -> str:
    try:
        ln = int(ln)
    except ValueError:
        return ln
    
    for section in SECTION_RANGES:
        if ln in SECTION_RANGES.get(section):
            return section
    
    return "OTHER"

## Helpers

In [17]:
def get_content(extension, file_obj):
    if extension == ".pdf":
        reader = fitz.open(stream=file_obj)
        return reader.load_page(0).get_text()
    elif extension == ".xlsx":
        return pd.read_excel(file_obj).to_string()
    elif extension == ".xlsb":
        return pd.read_excel(file_obj, engine='pyxlsb').to_string()
    else:
        return None

In [18]:
def replaced(_list:list, idxs:list, values:list) -> list:
    ret_list = _list.copy()
    for idx, value in zip(idxs, values):
        ret_list[idx] = value

    return ret_list

In [19]:
def find_outliers_iqr(SERIES, threshold=1.5):
    q1 = SERIES.quantile(0.25)
    q3 = SERIES.quantile(0.75)
    iqr = q3 - q1

    cutoff = threshold * iqr
    lower_bound = q1 - cutoff
    upper_bound = q3 + cutoff

    outliers = SERIES[(SERIES < lower_bound) | (SERIES > upper_bound)]

    return outliers


In [20]:
def get_row_idx(_df:pd.DataFrame, key:str) -> int:
    try:
        return (_df == key).any(axis=1).idxmax()
    except ValueError:
        return 0

In [21]:
def camelot_read_pdf_bytes(file_obj, table_num=0) -> pd.DataFrame:
    with tempfile.NamedTemporaryFile(suffix=".pdf", delete=False) as temp_pdf:
        temp_pdf.write(file_obj)
        return camelot.read_pdf(temp_pdf.name)._tables[table_num].df.copy()

In [22]:
def read_sheet(file_obj, extension:str) -> pd.DataFrame:
    if extension == ".xlsx":
        _df = pd.read_excel(file_obj)
    elif extension == ".xlsb":
        _df = pd.read_excel(file_obj, engine='pyxlsb')
    
    start = get_row_idx(_df, "LINE")
    if not "ACTUAL" in _df.iloc[start]:
        _df.columns = _df.iloc[start].fillna(_df.iloc[start-1])
        end = _df[start:].isna().all(axis=1).idxmax()
        _df = _df.iloc[start+1 : end]
    else:
        _df.columns = _df.iloc[start]
        _df = _df.iloc[start+1]

    _df.dropna(subset=["LINE", "PAYEE"], inplace=True)
    _df = _df.replace(["\)", ","], "", regex=True).replace("\(", "-", regex=True)
    _df.ACTUAL = pd.to_numeric(_df.ACTUAL, errors="coerce").astype(float)
    if "RATE" in _df.columns:
        _df.RATE = _df.RATE.astype(float)
    
    return _df

In [23]:
def find_outliers_iqr(SERIES, threshold=1.5):
    q1 = SERIES.quantile(0.25)
    q3 = SERIES.quantile(0.75)
    iqr = q3 - q1

    cutoff = threshold * iqr
    lower_bound = q1 - cutoff
    upper_bound = q3 + cutoff

    outliers = SERIES[(SERIES < lower_bound) | (SERIES > upper_bound)]

    return outliers


## READ COST SUMMARY

In [24]:
HB_CS_COLS = ["SECTION", "drop", "BID TOTALS", "ACTUAL", "VARIANCE"]


def read_hot_budget_cs(file_obj, extension) -> pd.DataFrame:
    if extension == ".pdf":
        _df = camelot_read_pdf_bytes(file_obj, 1)
        
        _df.drop(12, inplace=True)

        _df.columns = HB_CS_COLS
        _df.drop(columns=["drop"], inplace=True)
        _df = _df.loc[1:]

        _df = _df.replace([r"CS\d+\b ", r".*\n", "\)"], "", regex=True).replace("\(", "-", regex=True)

        _df[_df.columns[1:]] = _df.iloc[:, 1:].replace("", np.nan).apply(lambda x: x.str.replace(',', '')).astype(float)

        _df = _df.dropna(thresh=2)

        return _df.reset_index(drop=True)
    elif extension == ".xlsx":
        _df = pd.read_excel(file_obj)

        date_pattern = r'[A-Za-z]+\s+\d{1,2},\s+\d{4}'
        date_match = re.search(date_pattern, _df.columns[0])

        # Extract the matched date
        if date_match:
            date = pd.to_datetime(date_match.group(0))
        else:
            date = np.nan


        start = get_row_idx(_df, "ESTIMATED COST SUMMARY")
        _df.columns = _df.iloc[start]
        _df = _df.iloc[start+1: start + 24]

        dir_cost = get_row_idx(_df, "Direct Costs A - K")
        if dir_cost:
            _df.drop(dir_cost, inplace=True)
        
        _df = _df.dropna(how="all", axis=1).drop(11).dropna(thresh=3).rename(columns={"ESTIMATED COST SUMMARY":"SECTION"})
        _df.drop(_df.columns[1], axis=1, inplace=True)

        sep_nums = lambda x: x[re.search(r"\d ", x).end():]
        _df.SECTION = _df.SECTION.apply(sep_nums)
        _df["DATE"] = str(date)

        return _df.reset_index(drop=True)
    else:
        return pd.DataFrame()


def read_GetActual_cs(file_obj) -> pd.DataFrame:
    reader = fitz.open(stream=file_obj)
    content = reader.load_page(0).get_text()

    start = re.search(r"\b[A-Z]\s", content[2:]).start()
    content = re.sub(r"\b[A-Z]\s|Bid Actual|\,|\)", "", content.replace("(", "-"))
    content = content[start:content.find("\nGRAND TOTAL")].split("\n")
    _df = pd.DataFrame(columns=["SECTION", "BID TOTALS", "ACTUAL"])
    
    for line in content:
        vals = line.split("$")
        if len(vals) > 1:
            _df.loc[len(_df)] = vals[:3]

    _df[["BID TOTALS", "ACTUAL"]] = _df[["BID TOTALS", "ACTUAL"]].astype(float)
    _df = _df.drop(_df[_df.SECTION.str.contains("SUB TOTAL")].index)

    _df["VARIANCE"] = _df["ACTUAL"] - _df["BID TOTALS"]
    _df.SECTION = _df.SECTION.apply(str.strip)

    return _df

def clean_SECTION(val:str) -> str:
    val = val.strip()

    if "Production Fee" in val:
        val = "Production Fee"
    elif "Insurance" in val:
        val = "Insurance"
    elif "Talent Exp r" in val:
        val = "Talent Expenses"
    
    return val.upper()

def read_cost_summary(file_obj, extension) -> pd.DataFrame:
    content = get_content(extension, file_obj)
    
    if "ESTIMATED COST SUMMARY" in content:
        _df = read_hot_budget_cs(file_obj, extension)
    elif "Film Production Cost Summary" in content:
        _df = read_GetActual_cs(file_obj)
    else:
        return pd.DataFrame()
    
    _df.fillna(0, inplace=True)
    _df.SECTION = _df.SECTION.apply(clean_SECTION)
    
    _df["VARIANCE (%)"] = _df["VARIANCE"] / (_df["BID TOTALS"] + 1E-5) * 100

    for section in _df["SECTION"].unique():
        section_df = _df[_df["SECTION"] == section]
        outliers = find_outliers_iqr(section_df["VARIANCE (%)"])
        _df.loc[outliers.index, "VARIANCE (%)"] = section_df["VARIANCE (%)"].median()

    return _df

In [25]:
cs_excel = "Actual_Finish Line_SX22016.xlsx"
cs_pdf = "Actual_Finish Line_SX22016.pdf"

SECTION_DF_COLUMNS = ["LINE", "SUB SECTION", "DAYS", "RATE", "ESTIMATE", "ACTUAL"]


def clean_xlsx_section_df(section_df, section) -> pd.DataFrame:
    start = get_row_idx(section_df, section)
    section_df.columns = section_df.iloc[start]
    section_df = section_df.iloc[start+1:].reset_index(drop=True)
    section_df = section_df[:get_row_idx(section_df, "SUB TOTAL")]
    
    section_df = section_df[replaced(SECTION_DF_COLUMNS, [0, 1], [section_df.columns[0], section])]
    section_df.columns = SECTION_DF_COLUMNS
    section_df = section_df.dropna(thresh=3).reset_index(drop=True).fillna(0.0)
    
    section_df.ACTUAL = pd.to_numeric(section_df.ACTUAL, errors="coerce")
    section_df.dropna(inplace=True)
    
    section_df.insert(0, "SECTION", section)
    section_df["VARIANCE"] = section_df["ACTUAL"] - section_df["ESTIMATE"]
    section_df["VARIANCE (%)"] = section_df["VARIANCE"] / (section_df["ESTIMATE"] + 1E-5) * 100

    outliers = find_outliers_iqr(section_df["VARIANCE (%)"])
    section_df.loc[outliers.index, "VARIANCE (%)"] = section_df["VARIANCE (%)"].median()

    return section_df

def get_HB_xlsx_secion_dfs(cs, file_obj) -> pd.DataFrame:
    section_dfs = []
    _df = pd.read_excel(file_obj, header=37)

    for section in cs.SECTION.unique():
        try:
            section_dfs.append(clean_xlsx_section_df(_df.copy(), section))
        except:
            continue

    return pd.concat(section_dfs, ignore_index=True)

## READ SUB-SECTIONS

In [26]:
HB_PDF_SECTION_LOCS = {
    'PRE-PRODUCTION | WRAP LABOR': (1, 0),
    'SHOOTING LABOR': (2, 0),
    'PRE-PRODUCTION | WRAP EXPENSES': (3, 0),
    'LOCATION AND TRAVEL': (3, 1),
    'MAKEUP, WARDROBE, AND ANIMALS': (3, 2),
    'STUDIO | STAGE RENTAL / EXPENSES': (4, 0),
    'ART DEPARTMENT LABOR': (4, 1),
    'ART DEPARTMENT EXPENSES': (4, 2),
    'EQUIPMENT COSTS': (5, 0),
    'FILMSTOCK, DEVELOP AND PRINT': (5, 1),
    'MISCELLANEOUS': (5, 2),
    'DIRECTOR | CREATIVE FEES': (5, 3),
    'TALENT LABOR': (6, 0),
    'TALENT EXPENSES': (6, 1),
    'POST PRODUCTION LABOR': (7, 0),
    'EDITORIAL | FINISHING | POST PRODUCTION': (7, 1),
}

def to_read(sections:list):
    _to_read = {}

    for section in sections:
        info = HB_PDF_SECTION_LOCS.get(section)
        if not info:
            continue
        page = info[0]
        table = info[1]

        if not page in _to_read:
            _to_read[page] = []
        
        _to_read.get(page).append(table)
    
    return _to_read

def clean_pdf_section_df(section_df) -> pd.DataFrame:
    start = get_row_idx(section_df, "ACTUAL") or 0
    section_df.columns = section_df.iloc[start]
    section_df = section_df.iloc[start+1:].reset_index(drop=True)
    section_df = section_df[:get_row_idx(section_df, "SUB TOTAL")]
    section = section_df.columns[1]

    section_df = section_df[replaced(SECTION_DF_COLUMNS, [0, 1], [section_df.columns[0], section])]
    section_df.columns = SECTION_DF_COLUMNS

    section_df["ESTIMATE"] = section_df["ESTIMATE"].str.replace(',', '')
    section_df["ACTUAL"] = section_df["ACTUAL"].str.replace(',', '')

    section_df[["DAYS", "RATE", "ESTIMATE", "ACTUAL"]] = section_df[["DAYS", "RATE", "ESTIMATE", "ACTUAL"]].apply(pd.to_numeric, errors="coerce")
    section_df = section_df.dropna(subset="ACTUAL").reset_index(drop=True).fillna(0.0)

    section_df.insert(0, "SECTION", section)
    section_df["VARIANCE"] = section_df["ACTUAL"] - section_df["ESTIMATE"]
    section_df["VARIANCE (%)"] = section_df["VARIANCE"] / (section_df["ESTIMATE"] + 1E-5) * 100

    outliers = find_outliers_iqr(section_df["VARIANCE (%)"])
    section_df.loc[outliers.index, "VARIANCE (%)"] = section_df["VARIANCE (%)"].median()

    return section_df

def get_HB_pdf_secion_dfs(cs, file_obj):
    section_dfs = []

    with tempfile.NamedTemporaryFile(suffix=".pdf", delete=False) as temp_pdf:
        temp_pdf.write(file_obj)
        for page_num, table_nums in to_read(cs.SECTION.unique()).items():
            for table in camelot.read_pdf(temp_pdf.name, pages=str(page_num))._tables:
                if table.order in table_nums:
                    section_dfs.append(clean_pdf_section_df(table.df))

    return pd.concat(section_dfs, ignore_index=True)

In [27]:
def get_CS_section_dfs(cs, file_obj, extension) -> pd.DataFrame:
    section_dfs = None

    if "xlsx" in extension:
        section_dfs = get_HB_xlsx_secion_dfs(cs, file_obj)
    elif "pdf" in extension:
        section_dfs = get_HB_pdf_secion_dfs(cs, file_obj)
    else:
        return pd.DataFrame()
    
    return section_dfs.replace(r"\s{2,}.*", "", regex=True)

## TESTING

In [28]:
path = "/Users/aaronbastian/Documents/Jupyter/Film-Production-Company-Budget-Analysis/test/Actual_Finish Line_SX22016.xlsx"
with open(path, "rb") as file:
    ext = "." + path.split(".")[-1]
    file = file.read()

    cs = read_cost_summary(file, ext)


cs

9,SECTION,BID TOTALS,ACTUAL,VARIANCE,DATE,VARIANCE (%)
0,PRE-PRODUCTION | WRAP LABOR,45694.00,38888.46,-6805.54,2022-03-30 00:00:00,-1.489373e+01
1,PRE-PRODUCTION | WRAP EXPENSES,820.00,781.76,-38.24,2022-03-30 00:00:00,-4.663415e+00
2,LOCATION AND TRAVEL,4475.00,4805.97,330.97,2022-03-30 00:00:00,7.395978e+00
3,ART DEPARTMENT LABOR,16368.00,2515.00,-13853.00,2022-03-30 00:00:00,-8.463465e+01
4,ART DEPARTMENT EXPENSES,9000.00,7650.00,-1350.00,2022-03-30 00:00:00,-1.500000e+01
5,EQUIPMENT COSTS,13350.00,22147.20,8797.20,2022-03-30 00:00:00,6.589663e+01
6,"FILMSTOCK, DEVELOP AND PRINT",660.00,683.48,23.48,2022-03-30 00:00:00,3.557576e+00
7,MISCELLANEOUS,2250.00,3325.00,1075.00,2022-03-30 00:00:00,4.777778e+01
8,TALENT EXPENSES,0.00,396.90,396.90,2022-03-30 00:00:00,3.969000e+09
9,INSURANCE,4088.94,4088.94,0.00,2022-03-30 00:00:00,0.000000e+00
